In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import numpy as np
from kalman_estimation import Kalman4FROLS, Selector, get_mat_data
from tqdm import tqdm
from utils import get_term_dict

In [3]:
# !非线性模型
# *非线性数据
terms_path = '../data/nonlinear_terms10D_0.50trial24.mat'
term = Selector(terms_path)
_ = term.make_terms()

# # *保存候选项集合
# # fname = './data/nonlinear_candidate_terms.txt'
# fname = './data/longlag_nonlinear_candidate_terms.txt'
# np.savetxt(fname, terms_repr, fmt='%s')

# *selection
normalized_signals, Kalman_H, candidate_terms, Kalman_S_No = term.make_selection()

# *构造 Kalman Filter
kf = Kalman4FROLS(normalized_signals, Kalman_H=Kalman_H, uc=0.01)
y_coef = kf.estimate_coef()
print(y_coef)

[[ 1.29633307 -0.86805515 -0.02106495 -0.0179857   0.02315578]
 [ 0.2861246   0.75348293 -0.17453253 -0.13131189 -0.05715321]
 [ 0.97050607 -0.01302191 -0.01740498  0.00919405 -0.01371192]
 [-0.83600164  0.04372049  0.02801768  0.03222609 -0.04684601]
 [ 0.75993655 -0.34207697 -0.01440882  0.02245274  0.03588569]
 [-0.82717078  0.02582531  0.04520851  0.04286879 -0.044565  ]
 [-0.36560604  0.12962313  0.76069017 -0.14417012 -0.04903203]
 [-0.86640268  0.41510354  0.14220173  0.01059278  0.02096984]
 [-0.62079466  0.38881635  0.05099865  0.04296244 -0.04334044]
 [-0.89889762 -0.01902481  0.04616496  0.01866999  0.0240145 ]]


In [4]:
con_terms_linear5 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)', 'x1(t-3)', 'x1(t-2)', 'x4(t-1)', 'x5(t-1)', 'x4(t-1)', 'x5(t-1)']  # 9
con_terms_nonlinear5 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)*x1(t-2)', 'x1(t-3)', 'x1(t-2)*x1(t-2)', 'x4(t-1)', 'x5(t-1)', 'x4(t-1)', 'x5(t-1)']  # 9
true_coefs5 = [0.95*np.sqrt(2), -0.9025, 0.5, -0.4, -0.5, 0.25*np.sqrt(2), 0.25*np.sqrt(2), -0.25*np.sqrt(2), 0.25*np.sqrt(2)]  # 9
con_terms_linear10 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)', 'x2(t-3)', 'x1(t-2)', 'x4(t-4)', 'x9(t-2)', 'x4(t-4)', 'x1(t-1)', 'x1(t-2)', 'x7(t-2)', 
                      'x8(t-3)', 'x9(t-3)', 'x8(t-3)', 'x9(t-3)', 'x7(t-4)']  # 16
con_terms_nonlinear10 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)*x1(t-10)', 'x2(t-3)', 'x1(t-2)', 'x4(t-4)', 'x9(t-2)', 'x4(t-4)', 'x1(t-1)*x1(t-10)', 'x1(t-2)', 'x7(t-2)', 
                      'x8(t-3)', 'x9(t-3)', 'x8(t-3)', 'x9(t-3)', 'x7(t-4)']  # 16
true_coefs10 = [0.95*np.sqrt(2), -0.9025, 0.5, 0.9, -0.5, 0.8, -0.4, -0.8, 0.4, -0.4, -0.9, 0.4, 0.3, -0.3, 0.4, -0.75]  # 16
noises = np.linspace(0.5, 4, 8)
con_terms5 = [2, 1, 1, 3, 2]
con_terms10 = [2, 1, 1, 1, 2, 1, 2, 3, 2, 1]
root = '../data/'

In [5]:
term_dict1, term_dict2 = get_term_dict('nonlinear', 5)

- 形式参数
    - noise_var
    - trial
    - ndim
    - type
    - uc

In [6]:
def corr_term(y_coef, terms_set, Kalman_S_No, var_name: str = 'x', step_name: str = 't'):
    n_dim, n_term = y_coef.shape
    func_repr = []
    for var in range(n_dim):
        y = {}
        for term in range(n_term):
            y[terms_set[Kalman_S_No[var, term]]] = y_coef[var, term]
        func_repr.append(y)
    return func_repr

In [7]:
def frokf(noise_var, ndim, dtype, terms, length, root='../data/', trials=100, uc=0.01):
    assert dtype in ['linear', 'nonlinear'], 'type not support!'
    ax = []
    for trial in tqdm(range(1, trials + 1)):
        terms_path = root + f'{dtype}_terms{ndim}D_{noise_var:2.2f}trial{trial}.mat'
        term = Selector(terms_path)
        _ = term.make_terms()
        normalized_signals, Kalman_H, candidate_terms, Kalman_S_No = term.make_selection()
        Kalman_S_No = np.sort(Kalman_S_No)
        kf = Kalman4FROLS(normalized_signals, Kalman_H=Kalman_H, uc=uc)
        y_coef = kf.estimate_coef()
        terms_set = corr_term(y_coef, candidate_terms, Kalman_S_No)
        flatten_coef, t = [], 0
        for i in range(ndim):
            tmp = []
            for k in terms[t:t+length[i]]:
                tmp.append(terms_set[i][k] if k in terms_set[i] else np.nan)
            flatten_coef.extend(tmp)
            t += length[i]
        ax.append(flatten_coef)
    return np.stack(ax)

In [8]:
np.zeros((5, 5))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [9]:
candidate_terms

array(['x1(t-1)', 'x1(t-2)', 'x1(t-3)', ..., 'x10(t-9)*x10(t-9)',
       'x10(t-9)*x10(t-10)', 'x10(t-10)*x10(t-10)'], dtype='<U19')

In [10]:
term_dict1['x1(t-2)*x1(t-2)']
term_dict2[100]

100

'x1(t-2)*x1(t-2)'

In [12]:
frokf(2.5, 5, 'linear', con_terms_linear5, con_terms5, uc=1e-3)

100%|█████████████████████████████████████████████████████████████████| 100/100 [01:20<00:00,  1.26it/s]


array([[ 1.31804714, -0.89918689,  0.86447522, -0.76171028, -0.67281637,
         0.30359587,  0.20240677, -0.57953381,  0.32983637],
       [ 1.33027402, -0.89057117,  0.85214285, -0.67596926, -0.63892867,
         0.35301754,  0.18821179, -0.59376404,  0.34703589],
       [ 1.30427419, -0.8816175 ,  0.85013615, -0.83375033, -0.65423418,
         0.36255879,  0.2199043 , -0.55013924,  0.39042367],
       [ 1.31102987, -0.86454834,  0.85429741, -0.7763056 , -0.6054288 ,
         0.33999533,  0.20941492, -0.61299573,  0.33919083],
       [ 1.34301645, -0.9174802 ,  0.86949898, -0.72192555, -0.62801855,
         0.34326811,  0.22340898, -0.54750595,  0.363225  ],
       [ 1.36660385, -0.89100385,  0.84017651, -0.80689792, -0.65563255,
         0.32302995,  0.21035859, -0.56080681,  0.33805818],
       [ 1.31847224, -0.87603332,  0.83065116, -0.87502164, -0.63487054,
         0.32735257,  0.17362554, -0.60183918,  0.32871864],
       [ 1.36381388, -0.9078813 ,  0.86679121, -0.84936479, -0

In [ ]:
np.array(con_terms_linear10)

In [ ]:
np.empty((5,5), dtype='<U9')